# Winkler score (also called interval score)

An alternative way to assess prediction interval coverage for a specific time series is using the Winkler score.  This notebook contains code for the Winkler score as a measure of forecast accuracy of a prediction interval

The winkler score is defined as

$$W_{\alpha,t} = \begin{cases}
  (u_{\alpha,t} - \ell_{\alpha,t}) + \frac{2}{\alpha} (\ell_{\alpha,t} - y_t) & \text{if } y_t < \ell_{\alpha,t} \\
  (u_{\alpha,t} - \ell_{\alpha,t})   & \text{if }  \ell_{\alpha,t} \le y_t \le u_{\alpha,t} \\
  (u_{\alpha,t} - \ell_{\alpha,t}) + \frac{2}{\alpha} (y_t - u_{\alpha,t}) & \text{if } y_t > u_{\alpha,t}.
  \end{cases}$$
  
  
Where 

* $u_{\alpha, t}$ is the upper prediction interval value for $\alpha$ and horizon $t$
* $l_{\alpha, t}$ is the lower prediction interval value for $\alpha$ and horizon $t$
* $y_t$ is the ground truth observation at horizon $t$

**sources**

* https://otexts.com/fpp3/distaccuracy.html
* https://www.tandfonline.com/doi/abs/10.1198/016214506000001437
* https://www.tandfonline.com/doi/pdf/10.1080/01621459.1972.10481224?needAccess=true

## Interpretation

* A Winkler score is the width of the interval plus a penality proportional to the deviation (above or below the interval) and 2/$\alpha$
* Smaller winkler scores are better!

## Imports

In [4]:
import pandas as pd
import numpy as np

from forecast_tools.baseline import SNaive, Naive1
from forecast_tools.metrics import winkler_score
from forecast_tools.datasets import load_emergency_dept

import warnings
warnings.filterwarnings('ignore')

## Winkler score for an individual obs

Assume our model has generated a prediction interval of [744.54, 773.22].  We know that the true observation lies outside of the lower interval 741.84.  The winkler score is therefore:

In [7]:
alpha = 0.2
interval = [744.54, 773.22]
y_t = 741.84
ws = winkler_score(interval, y_t, alpha)
print(round(ws, 2))

55.68


if instead our y_t fell within the interval.  The winkler score would be equal to the width of the interval.

In [12]:
interval = [744.54, 773.22]
y_t = 745.0
ws = winkler_score(interval, y_t, alpha)
print(f'Winkler score: {round(ws, 2)}; Interval width: {round(interval[1]-interval[0], 2)}')

Winkler score: 28.68; Interval width: 28.68


## Winkler score for multiple step prediction

More commonly a model will generate a prediction over a multi-step horizon.  Here for example Seasonal naive is used to predict a 7 day holdout sample in the ED dataset.

In [16]:
HOLDOUT = 7
PERIOD = 7

attends = load_emergency_dept()

# train-test split
train, test = attends[:-HOLDOUT], attends[-HOLDOUT:]

model = SNaive(PERIOD)

# returns 80 and 90% prediction intervals by default.
preds, intervals = model.fit_predict(train, HOLDOUT, return_predict_int=True)

ws = winkler_score(intervals[0], test, alpha=0.2)
print(f'Winkler score: {ws:.2f}')

Winkler score: 130.75
